In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
spam = pd.read_csv('../input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv')
messages = spam.drop(labels = ["Clothing ID","Unnamed: 0","Age", "Title","Rating","Positive Feedback Count","Division Name","Class Name"], axis = 1)

messages.columns = ["text", "label","Department Name"]
messages['label'] = messages['label'].astype(str)
messages["label"] = messages["label"].str.replace("0", "negative")
messages["label"] = messages["label"].str.replace("1", "positive")
messages['text'] = messages['text'].astype('str') 
messages.head(10)
ss=messages.groupby('Department Name')
#botoms=ss.get_group('Bottoms')

#botoms.groupby('label').count()['Department Name'].plot(kind='pie',figsize=(8,6))

#plt.show()

#plot the scatter plot of balance and age variable in data
#sns.countplot(messages['Department Name'], palette='Blues')
#print(count(data['name']))


In [3]:
import plotly.express as px
fig = px.scatter(x=messages['Department Name'], y=messages['label'],labels={'x':'Departments', 'y':'sentiment'},title='Department wise')
fig.show()


In [4]:
import spacy


nlp = spacy.blank("en")


textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})


nlp.add_pipe(textcat)

In [5]:

textcat.add_label("positive")
textcat.add_label("negative")

1

In [6]:
train_texts = messages['text'].values
train_labels = [{'cats': {'positive': label == 'positive',
                          'negative': label == 'negative'}} 
                for label in messages['label']]

In [7]:
train_data = list(zip(train_texts, train_labels))
train_data[:4]

[('Absolutely wonderful - silky and sexy and comfortable',
  {'cats': {'positive': True, 'negative': False}}),
 ('Love this dress!  it\'s sooo pretty.  i happened to find it in a store, and i\'m glad i did bc i never would have ordered it online bc it\'s petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.',
  {'cats': {'positive': True, 'negative': False}}),
 ('I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c',
  {'cats': {'posit

In [8]:
from spacy.util import minibatch

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()


batches = minibatch(train_data, size=8)
for batch in batches:
   
    texts, labels = zip(*batch)
    nlp.update(texts, labels, sgd=optimizer)
    

In [9]:
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    
    batches = minibatch(train_data, size=8)
   
    for batch in batches:
        
        texts, labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 6.536790081999826}
{'textcat': 12.157932272961688}
{'textcat': 17.180891670630544}
{'textcat': 21.735206463566055}
{'textcat': 25.951043937352694}
{'textcat': 29.88449430133425}
{'textcat': 33.56117327386099}
{'textcat': 37.04402359821131}
{'textcat': 40.3249291833292}
{'textcat': 43.45373264055234}


In [10]:
texts = ["Are you ready for the tea party????? It's gonna be wild",
         "URGENT Reply to this message for GUARANTEED FREE TEA","You have won $50000" ,"HI lokesh, How are you?"]
docs = [nlp.tokenizer(text) for text in texts]
    

textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

[[0.8149355  0.18506446]
 [0.83635485 0.16364515]
 [0.81866485 0.18133518]
 [0.58217454 0.4178255 ]]


In [11]:

predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['positive', 'positive', 'positive', 'positive']
